# 성연령별 Recommender-system  
* 대분류별, 클러스터별

In [1]:
import numpy as np
import pandas as pd
import datetime as datetime
# import dataload
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
tqdm.pandas()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


# 0. 데이터 로드

In [2]:
cd /content/drive/MyDrive/2021/날씨공모전/code

/content/drive/MyDrive/2021/날씨공모전/code


### 0-1. `qty` and `cnt`

In [3]:
buy_full = pd.read_csv('../data/final_0620.csv', encoding='cp949')

In [4]:
buy_full['date'] = buy_full['date'].apply(lambda x : pd.to_datetime(x, format='%Y-%m-%d'))
buy_full['year'] = buy_full['date'].apply(lambda x : x.year)
buy_full['month'] = buy_full['date'].apply(lambda x : x.month)
buy_full['day'] = buy_full['date'].apply(lambda x : x.day)

In [5]:
buy_full.head()

,date,sex,big_cat,sm_cat,qty,ratio,오늘날씨,미세먼지,오늘 비,기상정보,태풍정보,cnt,avg_ta,max_ta,min_ta,rn_day,rn_hr1,avg_ws,PM10,PM25,hm_max,sum_ss_hr,max_pa,공휴일여부,주말여부,연_2019,월_2,월_3,월_4,월_5,월_6,월_7,월_8,월_9,월_10,월_11,월_12,분기_2,분기_3,분기_4,요일_1,요일_2,요일_3,요일_4,요일_5,요일_6,봄,여름,가을,겨울,age_20,age_30,age_40,age_50,year,month,day
0,2018-01-01,1,식품,가공란,37.0,0.0,10.69387,4.09137,0.00000,0.00000,0.04440,0.480964,0.885198,5.464146,-2.88074,0.0,0.0,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,2018,1,1
1,2018-01-01,1,식품,가공란,16.0,0.0,12.42969,6.35294,0.00000,0.51282,0.42417,0.480964,0.885198,5.464146,-2.88074,0.0,0.0,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,2018,1,1
2,2018-01-01,1,식품,가공란,9.0,0.0,13.02249,4.88037,0.00000,0.40650,0.00000,0.480964,0.885198,5.464146,-2.88074,0.0,0.0,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,2018,1,1
3,2018-01-01,1,식품,가공란,3.0,0.0,11.69080,4.53350,1.21951,0.90909,0.00000,0.480964,0.885198,5.464146,-2.88074,0.0,0.0,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,2018,1,1
4,2018-01-01,0,식품,가공란,13.0,0.0,11.23032,3.61296,0.38910,2.22222,0.04060,0.480964,0.885198,5.464146,-2.88074,0.0,0.0,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,2018,1,1


In [6]:
buy_full.shape

(2568668, 57)

### 0-2. corr 높은 품목 선택

In [7]:
adf_corr = pd.read_csv('../data/adf_corr_group_0617.csv', encoding='cp949')

In [20]:
buy_full = buy_full.merge(adf_corr[['sm_cat', 'group']], on='sm_cat')

In [28]:
data = buy_full[(buy_full.group=='정상_high')|(buy_full.group=='비정상_high')].reset_index(drop=True)

In [29]:
data.head()

,date,sex,big_cat,sm_cat,qty,ratio,오늘날씨,미세먼지,오늘 비,기상정보,태풍정보,cnt,avg_ta,max_ta,min_ta,rn_day,rn_hr1,avg_ws,PM10,PM25,hm_max,sum_ss_hr,max_pa,공휴일여부,주말여부,연_2019,월_2,월_3,월_4,월_5,월_6,월_7,월_8,월_9,월_10,월_11,월_12,분기_2,분기_3,분기_4,요일_1,요일_2,요일_3,요일_4,요일_5,요일_6,봄,여름,가을,겨울,age_20,age_30,age_40,age_50,year,month,day,group
0,2018-01-01,1,냉난방가전,가스온수기,0.0,10.52631,10.69387,4.09137,0.0000,0.00000,0.04440,0.480964,0.885198,5.464146,-2.88074,0.0,0.0,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,2018,1,1,정상_high
1,2018-01-01,1,냉난방가전,가스온수기,0.0,19.04761,12.42969,6.35294,0.0000,0.51282,0.42417,0.480964,0.885198,5.464146,-2.88074,0.0,0.0,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,2018,1,1,정상_high
2,2018-01-01,1,냉난방가전,가스온수기,0.0,16.66666,13.02249,4.88037,0.0000,0.40650,0.00000,0.480964,0.885198,5.464146,-2.88074,0.0,0.0,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,2018,1,1,정상_high
3,2018-01-01,1,냉난방가전,가스온수기,0.0,14.28571,13.92712,5.47887,0.0000,0.00000,0.00000,0.480964,0.885198,5.464146,-2.88074,0.0,0.0,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2018,1,1,정상_high
4,2018-01-01,0,냉난방가전,가스온수기,0.0,6.25000,11.23032,3.61296,0.3891,2.22222,0.04060,0.480964,0.885198,5.464146,-2.88074,0.0,0.0,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,2018,1,1,정상_high


# 1. 추천시스템 구현  
> 1) Collaborative Filtering - Item based  
> 2) Matrix Factorization

## 1-1. 데이터 선택  
ex) 6월, 20대 여성, 휴대용선풍기

In [79]:
final_data = data[(data.month==6)&(data.sex==1)&(data.age_20==1)]
final_data['date'] = final_data['date'].apply(lambda x : str(x)[:10])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [124]:
final_data['rn_day'].describe()

count    15326.000000
mean         4.356928
std          8.854612
min          0.000000
25%          0.000000
50%          0.096187
75%          3.399083
max         36.603354
Name: rn_day, dtype: float64

In [80]:
print(final_data.shape)
final_data.head()

(15326, 58)


,date,sex,big_cat,sm_cat,qty,ratio,오늘날씨,미세먼지,오늘 비,기상정보,태풍정보,cnt,avg_ta,max_ta,min_ta,rn_day,rn_hr1,avg_ws,PM10,PM25,hm_max,sum_ss_hr,max_pa,공휴일여부,주말여부,연_2019,월_2,월_3,월_4,월_5,월_6,월_7,월_8,월_9,월_10,월_11,월_12,분기_2,분기_3,분기_4,요일_1,요일_2,요일_3,요일_4,요일_5,요일_6,봄,여름,가을,겨울,age_20,age_30,age_40,age_50,year,month,day,group
1349,2018-06-01,1,냉난방가전,가스온수기,0.0,26.31578,29.54677,11.47619,0.79051,2.88461,0.04440,2.216935,21.610573,27.935125,15.236625,0.000000,0.000000,1.588177,42.772833,23.900980,85.267906,78.976417,971.692750,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,2018,6,1,정상_high
1357,2018-06-02,1,냉난방가전,가스온수기,0.0,5.26315,20.96899,9.24357,0.52700,0.00000,0.22202,2.251400,22.659667,29.110708,16.591042,0.000000,0.000000,1.678823,42.061861,23.881351,78.132656,77.510219,972.094542,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,2018,6,2,정상_high
1365,2018-06-03,1,냉난방가전,가스온수기,0.0,10.52631,13.19162,8.20610,0.65876,2.88461,0.04440,1.807984,22.244125,28.417615,16.699469,0.000000,0.000000,1.544750,40.569292,25.385581,77.587885,65.831708,970.339885,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,2018,6,3,정상_high
1373,2018-06-04,1,냉난방가전,가스온수기,0.0,5.26315,25.40047,10.30024,9.61791,4.80769,0.35523,1.868955,21.912000,27.211250,17.088917,0.000000,0.000000,1.718760,43.628463,27.122987,78.577708,53.567656,970.057094,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,2018,6,4,정상_high
1383,2018-06-05,1,냉난방가전,가스온수기,0.0,21.05263,25.29870,8.86587,9.35441,0.00000,0.17761,3.401936,21.268375,25.276510,18.063708,3.058031,0.780042,1.494458,43.269040,28.660388,85.717062,31.613000,969.456417,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,2018,6,5,정상_high


In [ ]:
final_data

In [81]:
item_user0 = pd.pivot_table(final_data, values='qty', index='sm_cat', columns='date').fillna(0)

In [82]:
print(item_user0.shape)
item_user0.head()

(263, 60)


date,2018-06-01,2018-06-02,2018-06-03,2018-06-04,2018-06-05,2018-06-06,2018-06-07,2018-06-08,2018-06-09,2018-06-10,2018-06-11,2018-06-12,2018-06-13,2018-06-14,2018-06-15,2018-06-16,2018-06-17,2018-06-18,2018-06-19,2018-06-20,2018-06-21,2018-06-22,2018-06-23,2018-06-24,2018-06-25,2018-06-26,2018-06-27,2018-06-28,2018-06-29,2018-06-30,2019-06-01,2019-06-02,2019-06-03,2019-06-04,2019-06-05,2019-06-06,2019-06-07,2019-06-08,2019-06-09,2019-06-10,2019-06-11,2019-06-12,2019-06-13,2019-06-14,2019-06-15,2019-06-16,2019-06-17,2019-06-18,2019-06-19,2019-06-20,2019-06-21,2019-06-22,2019-06-23,2019-06-24,2019-06-25,2019-06-26,2019-06-27,2019-06-28,2019-06-29,2019-06-30
sm_cat,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
가스온수기,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
가열식 가습기,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
가자미,0.0,1.0,2.0,3.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,3.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,2.0,2.0,4.0,1.0,1.0,0.0,1.0,3.0,2.0,1.0,1.0,1.0,1.0,0.0,5.0,2.0,0.0,3.0,0.0,0.0,1.0,1.0,2.0,0.0,1.0,1.0,3.0
갈비/찜/바비큐용 돈육,0.0,1.0,2.0,1.0,2.0,4.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,2.0,1.0,3.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,2.0,3.0,1.0,4.0,3.0,1.0,8.0,4.0,5.0,1.0,3.0,4.0,4.0,4.0,5.0,1.0,1.0,1.0,3.0,2.0,5.0,2.0,4.0,3.0,3.0,0.0,9.0,2.0,4.0,1.0,2.0,6.0,5.0,0.0,2.0,5.0
감/홍시,3.0,0.0,1.0,2.0,4.0,2.0,0.0,1.0,0.0,0.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,1.0,2.0,0.0,1.0,1.0,3.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,0.0


## 1-2. Collaborative Filtering - Item based   
* `buy_qty` 의 코사인 유사도 계산

In [64]:
from sklearn.metrics.pairwise import cosine_similarity

In [83]:
cosine_similarity0 = cosine_similarity(item_user0)
item0_collabor = pd.DataFrame(data=cosine_similarity0, index=item_user0.index, columns=item_user0.index)
print(item0_collabor.shape)
item0_collabor.head()

(263, 263)


sm_cat,가스온수기,가열식 가습기,가자미,갈비/찜/바비큐용 돈육,감/홍시,감귤/한라봉/오렌지,감말랭이,감자,갓김치,건강즙/녹용,건대추,건어물 멸치,건어물 진미채,건어물 황태,건자두,건포도,게장류,견과류,견과류 땅콩,견과류 마카다미아,견과류 밤,견과류 잣/은행,견과류 캐슈넛,견과류 호두,계란,고추/피망/파프리카,곡물차,공기정화 용품,공기청정기,곶감/반건시,과일류,과채 음료/주스,국내산 돈육,굴 생물,기능성 링클케어 화장품,기능성 아이케어 화장품,기능성 영양보습 화장품,기능성 화장품 세트,기초 화장용 로션,기초 화장용 미스트,...,키위/참다래,탁상/USB 선풍기,탄산수,탄산음료,태닝용 선크림,토마토,파/양파,파김치,팩도구,포도/거봉/체리,표고버섯,풋스프레이,풋워시/스크럽,풋크림,풋패치,프로바이오틱스,프로폴리스/로얄젤리,한우육,해조류 다시마,해조류 미역,해초류,핸드워시/스크럽,핸드크림,허브차,헤어 브러쉬,헤어스타일링용 펌제,헤어스타일링용 흑채,헤어스프레이,헤어에센스,헤어케어세트,호박,혼합견과,홍어,홍차,화장 비누,환자식,황토매트,회,휴대용 선풍기,히터
sm_cat,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
가스온수기,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
가열식 가습기,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
가자미,0.0,0.0,1.000000,0.626520,0.403025,0.728147,0.323048,0.639029,0.460575,0.558334,0.469342,0.727924,0.527793,0.635217,0.672100,0.488113,0.683642,0.655898,0.522853,0.729594,0.654247,0.584575,0.553687,0.593192,0.644606,0.694412,0.676490,0.712177,0.192007,0.348857,0.724055,0.652362,0.515197,0.299025,0.236795,0.661096,0.459455,0.416558,0.706108,0.669743,...,0.724350,0.657965,0.724248,0.509778,0.506634,0.687402,0.394980,0.528402,0.407572,0.660869,0.559230,0.512920,0.587289,0.711100,0.669396,0.721228,0.679941,0.650000,0.478824,0.662231,0.473611,0.719478,0.734218,0.658886,0.704737,0.625618,0.521532,0.727450,0.688414,0.586215,0.625597,0.590600,0.186436,0.732554,0.677594,0.635805,0.120824,0.712344,0.662329,0.427473
갈비/찜/바비큐용 돈육,0.0,0.0,0.626520,1.000000,0.384365,0.728089,0.360287,0.688311,0.513355,0.646904,0.455923,0.618132,0.512449,0.617994,0.687276,0.449266,0.751088,0.650526,0.617652,0.481401,0.570366,0.462476,0.614544,0.626469,0.717390,0.773539,0.684256,0.691210,0.227319,0.388044,0.746222,0.667303,0.604499,0.442522,0.314487,0.710604,0.643739,0.478068,0.743961,0.696922,...,0.753567,0.600600,0.757109,0.520808,0.412367,0.769409,0.569603,0.449292,0.400916,0.732547,0.537583,0.410067,0.679696,0.699119,0.754581,0.754466,0.682423,0.624554,0.413914,0.616202,0.530977,0.739761,0.756486,0.668787,0.721274,0.739456,0.600736,0.697618,0.753883,0.597096,0.642646,0.581056,0.266707,0.705244,0.727205,0.665082,0.000000,0.807754,0.612272,0.377010
감/홍시,0.0,0.0,0.403025,0.384365,1.000000,0.495006,0.379141,0.710087,0.439934,0.627979,0.496139,0.542271,0.400892,0.573727,0.486654,0.613515,0.447099,0.642971,0.461454,0.587298,0.556747,0.419532,0.525700,0.598813,0.617750,0.579383,0.529248,0.635044,0.165342,0.352506,0.560434,0.660428,0.577956,0.183450,0.312893,0.699349,0.438468,0.312925,0.675292,0.685276,...,0.615504,0.598277,0.589930,0.714649,0.411831,0.559185,0.370571,0.272986,0.588858,0.424518,0.457445,0.405096,0.582020,0.562419,

* 아이템 A와 코사인 유사도가 높은 10개의 아이템 도출

In [84]:
def get_item_based_collabor(sm_cat):
    return item0_collabor[sm_cat].sort_values(ascending=False)[1:12]

In [85]:
get_item_based_collabor('휴대용 선풍기')

sm_cat
바디케어용 제모제         0.952251
유제품 음료            0.948218
기초 화장용 크림         0.940870
바디 보습제            0.939468
베이스 메이크업용 컨실러     0.935645
선크림               0.935278
화장 비누             0.935205
기초 화장용 로션         0.935018
스킨케어 코팩           0.933788
레몬/자몽             0.933667
베이스 메이크업용 BB크림    0.933410
Name: 휴대용 선풍기, dtype: float64

## 1-3. Matrix Factorization  

### 1-3-1. 타겟 유저와 유사한 유저가 많이 소비한 상품을 추천 (유저기반)  
  
=> 아이템별로 추천이 나와야해서 얘는 못쓸듯

In [68]:
from sklearn.decomposition import TruncatedSVD

In [86]:
user_item0 = pd.pivot_table(final_data, values='qty', index='date', columns='sm_cat').fillna(0)
print(user_item0.shape)
user_item0.head()

(60, 263)


sm_cat,가스온수기,가열식 가습기,가자미,갈비/찜/바비큐용 돈육,감/홍시,감귤/한라봉/오렌지,감말랭이,감자,갓김치,건강즙/녹용,건대추,건어물 멸치,건어물 진미채,건어물 황태,건자두,건포도,게장류,견과류,견과류 땅콩,견과류 마카다미아,견과류 밤,견과류 잣/은행,견과류 캐슈넛,견과류 호두,계란,고추/피망/파프리카,곡물차,공기정화 용품,공기청정기,곶감/반건시,과일류,과채 음료/주스,국내산 돈육,굴 생물,기능성 링클케어 화장품,기능성 아이케어 화장품,기능성 영양보습 화장품,기능성 화장품 세트,기초 화장용 로션,기초 화장용 미스트,...,키위/참다래,탁상/USB 선풍기,탄산수,탄산음료,태닝용 선크림,토마토,파/양파,파김치,팩도구,포도/거봉/체리,표고버섯,풋스프레이,풋워시/스크럽,풋크림,풋패치,프로바이오틱스,프로폴리스/로얄젤리,한우육,해조류 다시마,해조류 미역,해초류,핸드워시/스크럽,핸드크림,허브차,헤어 브러쉬,헤어스타일링용 펌제,헤어스타일링용 흑채,헤어스프레이,헤어에센스,헤어케어세트,호박,혼합견과,홍어,홍차,화장 비누,환자식,황토매트,회,휴대용 선풍기,히터
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-06-01,0.0,0.0,0.0,0.0,3.0,14.0,0.0,39.0,1.0,12.0,0.0,1.0,0.0,4.0,8.0,3.0,10.0,22.0,3.0,3.0,3.0,1.0,1.0,5.0,13.0,14.0,2.0,15.0,12.0,0.0,16.0,51.0,4.0,0.0,0.0,46.0,2.0,0.0,52.0,87.0,...,7.0,8.0,21.0,128.0,1.0,22.0,13.0,1.0,7.0,0.0,0.0,0.0,8.0,2.0,10.0,21.0,8.0,7.0,0.0,4.0,1.0,6.0,16.0,18.0,9.0,12.0,2.0,7.0,46.0,2.0,18.0,5.0,0.0,5.0,26.0,0.0,0.0,179.0,59.0,3.0
2018-06-02,0.0,0.0,1.0,1.0,0.0,3.0,0.0,33.0,0.0,6.0,1.0,1.0,1.0,3.0,10.0,0.0,6.0,12.0,1.0,2.0,1.0,1.0,2.0,2.0,12.0,4.0,0.0,11.0,3.0,1.0,9.0,29.0,2.0,0.0,0.0,26.0,0.0,1.0,48.0,47.0,...,4.0,6.0,13.0,176.0,5.0,18.0,3.0,1.0,10.0,0.0,2.0,0.0,0.0,9.0,14.0,12.0,13.0,17.0,0.0,4.0,1.0,6.0,21.0,13.0,9.0,5.0,4.0,5.0,50.0,10.0,12.0,4.0,0.0,4.0,23.0,1.0,0.0,184.0,50.0,2.0
2018-06-03,0.0,0.0,2.0,2.0,1.0,3.0,0.0,34.0,4.0,5.0,2.0,1.0,1.0,4.0,8.0,3.0,9.0,10.0,1.0,3.0,7.0,0.0,0.0,2.0,9.0,3.0,2.0,11.0,3.0,0.0,8.0,61.0,0.0,1.0,0.0,43.0,1.0,0.0,64.0,41.0,...,8.0,9.0,19.0,155.0,7.0,20.0,13.0,0.0,11.0,1.0,1.0,2.0,4.0,5.0,17.0,20.0,14.0,18.0,2.0,2.0,0.0,6.0,32.0,27.0,17.0,5.0,2.0,9.0,49.0,1.0,12.0,8.0,0.0,9.0,44.0,3.0,0.0,173.0,60.0,0.0
2018-06-04,0.0,0.0,3.0,1.0,2.0,8.0,0.0,39.0,2.0,5.0,0.0,3.0,0.0,1.0,5.0,4.0,5.0,18.0,1.0,5.0,17.0,0.0,1.0,11.0,12.0,3.0,4.0,15.0,1.0,0.0,12.0,67.0,2.0,0.0,1.0,34.0,2.0,0.0,81.0,38.0,...,9.0,12.0,41.0,122.0,2.0,19.0,9.0,0.0,2.0,3.0,0.0,3.0,6.0,8.0,18.0,22.0,14.0,14.0,1.0,3.0,1.0,18.0,33.0,24.0,17.0,8.0,4.0,7.0,81.0,3.0,14.0,14.0,0.0,11.0,57.0,6.0,0.0,171.0,96.0,1.0
2018-06-05,0.0,0.0,0.0,2.0,4.0,6.0,0.0,35.0,6.0,6.0,0.0,2.0,2.0,2.0,6.0,2.0,4.0,16.0,0.0,1.0,3.0,0.0,2.0,3.0,12.0,5.0,4.0,9.0,1.0,0.0,15.0,41.0,5.0,0.0,1.0,40.0,2.0,1.0,103.0,48.0,...,11.0,6.0,13.0,125.0,1.0,13.0,15.0,0.0,5.0,1.0,0.0,1.0,7.0,2.0,14.0,26.0,11.0,9.0,13.0,1.0,2.0,7.0,20.0,22.0,12.0,10.0,2.0,6.0,85.0,3.0,24.0,16.0,0.0,4.0,32.0,2.0,0.0,205.0,83.0,1.0


In [87]:
# SVD (특이값 분해)
SVD = TruncatedSVD(n_components=10)
matrix = SVD.fit_transform(user_item0)
matrix.shape

(60, 10)

In [88]:
# 피어슨 상관계수
corr = np.corrcoef(matrix)
corr.shape

(60, 60)

In [90]:
user_list = list(user_item0.index)
corr_user = corr[user_list.index('2018-06-21')] # target user
user_recommend = list(user_item0.index[(corr_user>0.9)])[:50] # target user와 가장 비슷한 최대 50명 user

# 50명 user들이 가장 많이 소비한 item 10개 추천
final_recommend = list(user_item0[user_item0.index.isin(user_recommend)].sum().sort_values(ascending=False)[:10].index) 
final_recommend

['기초 화장용 크림',
 '회',
 '생수',
 '카페 푸드',
 '네일 메이크업 용품',
 '기초 화장용 에센스',
 '색조 메이크업 립스틱',
 '색조 메이크업 립틴트',
 '베이스 메이크업용 쿠션팩트',
 '탄산음료']

### 1-3-2. 타겟 상품과 유사한 상품 추천

In [91]:
# SVD (특이값 분해)
SVD = TruncatedSVD(n_components=10)
matrix = SVD.fit_transform(item_user0)
matrix.shape

(263, 10)

In [92]:
# 피어슨 상관계수
corr = np.corrcoef(matrix)
corr.shape

/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2559: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2560: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


(263, 263)

In [102]:
item_list = list(item_user0.index)
corr_item = corr[item_list.index('휴대용 선풍기')] # target item
item_recommend = list(item_user0.index[np.argsort(-corr_item)])[1:51] # target item과 가장 유사한 최대 50개 item
item_recommend

['유제품 음료',
 '벽걸이 에어컨',
 '혼합견과',
 '레몬/자몽',
 '과채 음료/주스',
 '바디케어용 제모제',
 '견과류',
 '베이스 메이크업용 BB크림',
 '프로폴리스/로얄젤리',
 '의류건조기',
 '탁상/USB 선풍기',
 '린스',
 '바디 보습제',
 '옻/칡/쑥즙',
 '스킨케어 코팩',
 '화장 비누',
 '허브차',
 '저/무지방우유',
 '아몬드유/코코넛밀크',
 '기초 화장용 미스트',
 '벽걸이형 선풍기',
 '베이스 메이크업용 컨실러',
 '기초 화장용 크림',
 '딸기/복분자/블루베리',
 '호박',
 '잡곡 씨드류',
 '아이스티',
 '견과류 호두',
 '배/포도/과일즙',
 '코코아/핫초코',
 '음용 식초',
 '업소용 선풍기',
 '차/곡물 음료',
 '전통차',
 '견과류 밤',
 '선크림',
 '이온음료',
 '기능성 아이케어 화장품',
 '건강즙/녹용',
 '칼슘/철분 영양제',
 '베이스 메이크업용 쿠션팩트',
 '메이크업 브러쉬',
 '헤어에센스',
 '녹차',
 '다이어트보조식',
 '클렌징 폼',
 '색조 메이크업 립스틱',
 '남성 쉐이빙',
 '기초 화장용 로션',
 '남성 스킨']

# 2. 함수화

## 2-1. 기준  
* 비안옴 / 비옴
* 미세먼지 좋음 / 나쁨

In [161]:
data.rn_day.describe()

count    1.785003e+06
mean     3.111790e+00
std      8.013387e+00
min      0.000000e+00
25%      0.000000e+00
50%      5.500000e-02
75%      1.391063e+00
max      7.218162e+01
Name: rn_day, dtype: float64

In [156]:
data.PM10.describe()

count    1.785003e+06
mean     3.759340e+01
std      1.843036e+01
min      8.096932e+00
25%      2.447371e+01
50%      3.304657e+01
75%      4.615175e+01
max      1.276624e+02
Name: PM10, dtype: float64

## 2-2. Collaborative Filtering - Item based

In [149]:
from sklearn.metrics.pairwise import cosine_similarity

class cf_item:
    def __init__(self, data, target, age, sex, month):
        self.data = data
        self.target = target
        self.age = age
        self.sex = sex
        self.month = month

    def recommender(self, number):
        # age
        if self.age == 20:
            final_data = self.data[self.data['age_20']==1]
        elif self.age == 30:
            final_data = self.data[self.data['age_30']==1]
        elif self.age == 40:
            final_data = self.data[self.data['age_40']==1]
        elif self.age == 50:
            final_data = self.data[self.data['age_50']==1]
        else:
            final_data = self.data[(self.data['age_20']==0)&(self.data['age_30']==0)&(self.data['age_40']==0)&(self.data['age_50']==0)]
        # sex
        if self.sex == 'F':
            final_data = final_data[final_data['sex']==1]
        elif self.sex == 'M':
            final_data = final_data[final_data['sex']==0]
        # month
        final_data = final_data[final_data['month']==self.month]

        # preprocessing
        final_data['date'] = final_data['date'].apply(lambda x : str(x)[:10])
        self.final_data = final_data
        
        # collaborative filtering
        item_user0 = pd.pivot_table(final_data, values='qty', index='sm_cat', columns='date').fillna(0)
        cosine_similarity0 = cosine_similarity(item_user0)
        item_collabor0 = pd.DataFrame(data=cosine_similarity0, index=item_user0.index, columns=item_user0.index)
        return item_collabor0[self.target].sort_values(ascending=False)[1:number]

    def recommender_weather(self, number, rain, dust):
        # 비안옴(0) & 미세먼지 좋음(0)
        if (rain == 0)&(dust == 0):
            weather_data = self.final_data[(self.final_data['rn_day']==0)&(self.final_data['PM10']<81)]
        # 비안옴(0) & 미세먼지 나쁨(1)    
        elif (rain == 0)&(dust == 1):
            weather_data = self.final_data[(self.final_data['rn_day']==0)&(self.final_data['PM10']>=81)]
        # 비옴(1) & 미세먼지 좋(0)    
        elif (rain == 1)&(dust == 0):
            weather_data = self.final_data[(self.final_data['rn_day']>0)&(self.final_data['PM10']<81)]
        # 비옴(1) & 미세먼지 나쁨(1)    
        elif (rain == 1)&(dust == 1):
            weather_data = self.final_data[(self.final_data['rn_day']>0)&(self.final_data['PM10']>=81)]
        
        if weather_data.shape[0] == 0:
            print('오늘 날씨에 해당되는 데이터가 없습니다.')

        else:
            # collaborative filtering
            item_user1 = pd.pivot_table(weather_data, values='qty', index='sm_cat', columns='date').fillna(0)
            cosine_similarity1 = cosine_similarity(item_user1)
            item_collabor1 = pd.DataFrame(data=cosine_similarity1, index=item_user1.index, columns=item_user1.index)
            return item_collabor1[self.target].sort_values(ascending=False)[1:number]

In [150]:
cf = cf_item(data, '휴대용 선풍기', 20, 'F', 6)

In [151]:
cf.recommender(10)

sm_cat
바디케어용 제모제        0.952251
유제품 음료           0.948218
기초 화장용 크림        0.940870
바디 보습제           0.939468
베이스 메이크업용 컨실러    0.935645
선크림              0.935278
화장 비누            0.935205
기초 화장용 로션        0.935018
스킨케어 코팩          0.933788
Name: 휴대용 선풍기, dtype: float64

In [158]:
cf.recommender_weather(10, 0, 0)

sm_cat
선케어용 선밤      0.958671
바디케어용 제모제    0.954944
카페 푸드        0.951688
데오드란트        0.950628
견과류          0.948602
바디 보습제       0.947951
과채 음료/주스     0.946884
기초 화장용 크림    0.946152
핸드크림         0.945390
Name: 휴대용 선풍기, dtype: float64

In [155]:
cf.recommender_weather(10, 0, 1)

오늘 날씨에 해당되는 데이터가 없습니다.


In [157]:
cf.recommender_weather(10, 1, 0)

sm_cat
바디케어용 제모제        0.957117
유제품 음료           0.956368
기초 화장용 크림        0.948545
기초 화장용 로션        0.947286
린스               0.946721
베이스 메이크업용 컨실러    0.946509
스킨케어 코팩          0.946213
선크림              0.939147
바디 보습제           0.939131
Name: 휴대용 선풍기, dtype: float64

In [160]:
cf.recommender_weather(10, 1, 1)

오늘 날씨에 해당되는 데이터가 없습니다.


## 2-3. Matrix Factorzation (타겟 상품과 유사한 상품 추천)

In [163]:
from sklearn.decomposition import TruncatedSVD

class mf_item:
    def __init__(self, data, target, age, sex, month):
        self.data = data
        self.target = target
        self.age = age
        self.sex = sex
        self.month = month

    def recommender(self, number):
        # age
        if self.age == 20:
            final_data = self.data[self.data['age_20']==1]
        elif self.age == 30:
            final_data = self.data[self.data['age_30']==1]
        elif self.age == 40:
            final_data = self.data[self.data['age_40']==1]
        elif self.age == 50:
            final_data = self.data[self.data['age_50']==1]
        else:
            final_data = self.data[(self.data['age_20']==0)&(self.data['age_30']==0)&(self.data['age_40']==0)&(self.data['age_50']==0)]
        # sex
        if self.sex == 'F':
            final_data = final_data[final_data['sex']==1]
        elif self.sex == 'M':
            final_data = final_data[final_data['sex']==0]
        # month
        final_data = final_data[final_data['month']==self.month]

        # preprocessing
        final_data['date'] = final_data['date'].apply(lambda x : str(x)[:10])
        self.final_data = final_data
        
        # Matrix Factorization
        item_user0 = pd.pivot_table(final_data, values='qty', index='sm_cat', columns='date').fillna(0)
        SVD0 = TruncatedSVD(n_components=10) # SVD (특이값 분해)
        matrix0 = SVD0.fit_transform(item_user0)
        corr0 = np.corrcoef(matrix0) # 피어슨 상관계수
        item_list0 = list(item_user0.index)
        corr_item0 = corr0[item_list0.index(self.target)] # target item
        return list(item_user0.index[np.argsort(-corr_item0)])[1:(number+1)] # target item과 가장 유사한 최대 number개 item

    def recommender_weather(self, number, rain, dust):
        # 비안옴(0) & 미세먼지 좋음(0)
        if (rain == 0)&(dust == 0):
            weather_data = self.final_data[(self.final_data['rn_day']==0)&(self.final_data['PM10']<81)]
        # 비안옴(0) & 미세먼지 나쁨(1)    
        elif (rain == 0)&(dust == 1):
            weather_data = self.final_data[(self.final_data['rn_day']==0)&(self.final_data['PM10']>=81)]
        # 비옴(1) & 미세먼지 좋(0)    
        elif (rain == 1)&(dust == 0):
            weather_data = self.final_data[(self.final_data['rn_day']>0)&(self.final_data['PM10']<81)]
        # 비옴(1) & 미세먼지 나쁨(1)    
        elif (rain == 1)&(dust == 1):
            weather_data = self.final_data[(self.final_data['rn_day']>0)&(self.final_data['PM10']>=81)]
        
        if weather_data.shape[0] == 0:
            print('오늘 날씨에 해당되는 데이터가 없습니다.')

        else:
            # Matrix Factorization
            item_user1 = pd.pivot_table(weather_data, values='qty', index='sm_cat', columns='date').fillna(0)
            SVD1 = TruncatedSVD(n_components=10) # SVD (특이값 분해)
            matrix1 = SVD1.fit_transform(item_user1)
            corr1 = np.corrcoef(matrix1) # 피어슨 상관계수
            item_list1 = list(item_user1.index)
            corr_item1 = corr1[item_list1.index(self.target)] # target item
            return list(item_user1.index[np.argsort(-corr_item1)])[1:(number+1)] # target item과 가장 유사한 최대 number개 item

In [164]:
mf = mf_item(data, '휴대용 선풍기', 20, 'F', 6)

In [165]:
mf.recommender(10)

/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2559: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2560: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


['유제품 음료',
 '벽걸이 에어컨',
 '혼합견과',
 '레몬/자몽',
 '과채 음료/주스',
 '바디케어용 제모제',
 '견과류',
 '베이스 메이크업용 BB크림',
 '프로폴리스/로얄젤리',
 '의류건조기']

In [166]:
mf.recommender_weather(10, 0, 0)

/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2559: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2560: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


['선케어용 선밤',
 '벽걸이 에어컨',
 '베이스 메이크업용 BB크림',
 '데오드란트',
 '견과류 호두',
 '공기정화 용품',
 '바디케어용 제모제',
 '유제품 음료',
 '화장 비누',
 '수산 생물']

In [167]:
mf.recommender_weather(10, 0, 1)

오늘 날씨에 해당되는 데이터가 없습니다.


In [168]:
mf.recommender_weather(10, 1, 0)

/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2559: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2560: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


['레몬/자몽',
 '유제품 음료',
 '허브차',
 '베이스 메이크업용 BB크림',
 '바디케어용 제모제',
 '기초 화장용 크림',
 '베이스 메이크업용 컨실러',
 '과채 음료/주스',
 '녹차',
 '프로폴리스/로얄젤리']

In [169]:
mf.recommender_weather(10, 1, 1)

오늘 날씨에 해당되는 데이터가 없습니다.
